<a href="https://colab.research.google.com/github/eminshall/Notebooks-Youtube-Channel/blob/main/Estimating_Value_at_Risk_on_Crypto_Portfolio_Using_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
risky_assets = ['BTC-USD','ETH-USD','BNB-USD']
share = [1,5,5]
start = '2024-01-01'
end = '2024-12-31'
T = 1
n_sims = 10**5

In [ ]:
df = yf.download(risky_assets,start=start,end=end)['Close']

returns = df['Close'].pct_change().dropna()
returns.plot(title='Exxon and Chevron daily stock returns in 2024')

In [ ]:
# Create a covariance matrix
cov_mat = returns.cov()

In [ ]:
# Cholesky decomposition of covariance returns
chol_mat = np.linalg.cholesky(cov_mat)

In [ ]:
# Draw the correlated random numbers from the Standard Normal Distribution
rv = np.random.normal(size=(n_sims, len(risky_assets)))
correlated_rv = np.transpose(np.matmul(chol_mat, np.transpose(rv)))

In [ ]:
# Simulate price with monte carlo
r = np.mean(returns, axis=0).values
sigma = np.std(returns, axis=0).values
s_0 = df['Close'].values[-1, :]
p_0 = np.sum(share * s_0)

s_t = s_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * correlated_rv)
p_t = np.sum(share * s_t, axis=1)
p_diff = p_t - p_0

In [ ]:
p_diff_sorted = np.sort(p_diff)
percentiles = [0.01, 0.1, 1.0]
var = np.percentile(p_diff, q=percentiles)

for x, y in zip(percentiles, var):
  print(f'1-Day VaR with {100-x}% Confidence: ${-y: .2f}')

In [ ]:
ax = sns.distplot(p_diff, kde=False)
ax.set_title('Distribution of possible 1-Day Changes in Portfolio Value 1-Day 99% VaR')
ax.axvline(var[2],0,10000)